In [ ]:
import os
from tensorboard.backend.event_processing.event_accumulator import EventAccumulator
import pandas as pd

# Function to extract scalar values from TensorBoard logs
def extract_scalars_from_folder(folder, up_to_step=None):
    scalar_data = {}

    for root, dirs, files in os.walk(folder):
        for file in files:
            if file.startswith("events.out.tfevents"):
                log_path = os.path.join(root, file)

                # Load TensorBoard logs
                event_acc = EventAccumulator(log_path)
                event_acc.Reload()

                # Extract scalar values
                for tag in event_acc.Tags().get("scalars", []):
                    if tag not in scalar_data:
                        scalar_data[tag] = []

                    for scalar_event in event_acc.Scalars(tag):
                        if up_to_step is None or scalar_event.step <= up_to_step:
                            scalar_data[tag].append(scalar_event.value)

    # Compute mean for each scalar
    mean_values = {tag: sum(values) / len(values) for tag, values in scalar_data.items() if values}
    return mean_values

# Function to process multiple folders
def extract_scalars_from_folders(folders, up_to_step=None):
    all_results = {}
    for folder in folders:
        print(f"Processing folder: {folder}")
        all_results[folder] = extract_scalars_from_folder(folder, up_to_step)
    return all_results

# Function to format the scalar data into a structured table
def print_structured_data(scalar_data):
    all_dataframes = []
    for folder, scalars in scalar_data.items():
        df = pd.DataFrame.from_dict(scalars, orient="index", columns=["Mean Value"])
        df.index.name = "Scalar Name"
        df["Folder"] = folder
        all_dataframes.append(df)

    result_df = pd.concat(all_dataframes)
    print(result_df)

# List of TensorBoard log folders
folders = ["/home/albert/github/embodiment-scaling-law-sim2real/log_dir/test_2",
           "/home/albert/github/embodiment-scaling-law-sim2real/log_dir/test_3090_2",
           "/home/albert/github/embodiment-scaling-law-sim2real/log_dir/test_4090_2"]  # Replace with your actual log directories

# Specify the timestep up to which you want to calculate the mean
up_to_step = 70000  # Replace with your desired step limit

# Extract and process the scalar data
scalar_means = extract_scalars_from_folders(folders, up_to_step)

# Print the results in a structured format
print_structured_data(scalar_means)


In [ ]:
scalar_means